In [ ]:
# pip install -r .\requirments.txt
import re
import nltk
import numpy as np
import pandas as pd

In [ ]:
# ==================== Обработка исходных данных ==================== #
filename = "spam.csv"
data = pd.read_csv(filename, encoding='latin-1')
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v2" : "text", "v1":"label"})
data.head()

In [ ]:
# ==================== Токенизация ==================== #
def tokenize(method):
    ham_tokens = np.ndarray(0)
    spam_tokens = np.ndarray(0)
    
    for i in range(data.index.start, data.index.stop):
        tokens = method(data.loc[i]["text"])
        if len(tokens) != 0:
            lower_case_tokens = np.char.lower(tokens)
            if data.loc[i]["label"] == "ham":
                ham_tokens = np.concatenate((ham_tokens, lower_case_tokens))
            else:
                spam_tokens = np.concatenate((spam_tokens, lower_case_tokens))
    
    return  ham_tokens, spam_tokens


method = "nltk" if 1 else "regex"
match method: 
    case "nltk":
        # Punkt Tokenizer
        # nltk.download("punkt")
        ham_tokens, spam_tokens = tokenize(nltk.word_tokenize)
    case "regex":
        # Регулярные выражения
        regural_tokenize = lambda text: re.findall('\w+', text)
        ham_tokens, spam_tokens = tokenize(regural_tokenize)

In [ ]:
# ==================== Удаление стоп-слов ==================== #
# nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words('english')

is_stop_words = lambda token: True if token in stop_words else False
v_is_stop_words = np.vectorize(is_stop_words)
ham_tokens_w_sw = ham_tokens[v_is_stop_words(ham_tokens) == False]
spam_tokens_w_sw = spam_tokens[v_is_stop_words(spam_tokens) == False]

print("HAM: \tЧисло токенов: ", ham_tokens.shape[0], "Число токенов без стоп-слов: ", ham_tokens_w_sw.shape[0])
print("SPAM: \tЧисло токенов: ", spam_tokens.shape[0], "Число токенов без стоп-слов: ", spam_tokens_w_sw.shape[0])

In [ ]:
# ==================== Удаление знаков препинания ==================== #
import string
ham_tokens_w_sw = ham_tokens_w_sw.translate(None, string.punctuation)

In [ ]:
nlp_words=nltk.FreqDist(ham_tokens_w_sw)
nlp_words.plot(30);